# 환경 설정

In [ ]:
!pip install numpy==1.22.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 48.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.22.1 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but you have numpy 1.22.1 which is incompatible.
librosa 0.10.2.post1 requires numpy!=1.22.0,!=1.22.1,!=1.22.2,>=1.20.3, but you have numpy 1.22.1 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.22.1 which is incompatible.
plotnine 0.12.4 requires numpy>=1.23.0, but you have numpy 1.22.1 which is incompatible.
pywavelets 1.6.0 requires numpy<3,>=1.22.4, but you have numpy 1.22.1 which is 

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch>=1.8.1

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Using cached mxnet-1.9.1-py3-none-manylinux2014_x86_64.whl (49.1 MB)
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-01ynkr49/kobert-tokenizer_1f755b9515324c66aad88870ca16df60
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-01ynkr49/kobert-tokenizer_1f755b9515324c66aad88870ca16df60
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=e44668dc8782318752c38b1fb2a2e863d7f7ead38116cf3a6befde939188e4f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-7lztmx2e/wheels/e9/1a/3f/a864970e8a169c176befa3c4a

In [ ]:
import gluonnlp as nlp


/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


# 데이터 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 경로 설정
path = '/content/drive/MyDrive/'

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# Torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/annotation_1.xlsx - Sheet1.csv')

In [ ]:
data.head()

,id,NO1,Column
0,1,0,내담자: 안녕하세요 상담선생님. 저는 오랫동안 힘든 시간을 보냈습니다.
1,1,0,내담자: 초등학교 때 왕따를 심하게 당해 중도에 학교를 그만뒀어요. 그 후 정신과 ...
2,1,0,내담자: 직장생활도 그렇게 오래 유지하지 못했어요. 몇 개월씩만 다녔죠. 그리고 이...
3,1,0,내담자: 요즘에는 버스에서도 옆자리에 아무도 앉지 않더라고요. 청소년 문화인가 싶어...
4,1,0,내담자: 늦은 나이에 직장도 없이 '조현병' 진단까지 받았습니다. 이성에 대한 호기...


In [ ]:
#data['class'] = [1 if s >=3 else 0 for s in data['rating']] # 긍정이면 1, 부정이면 0
data.drop_duplicates(subset = ['Column'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
data['Column'] = data['Column'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
data['Column'] = data['Column'].str.replace('^ +', "") # 공백은 empty 값으로 변경
data['Column'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
data = data.dropna(how='any') # Null 값 제거

In [ ]:
data['NO1'][data.NO1 == 2] = 1

<ipython-input-11-9b02a7944164>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NO1'][data.NO1 == 2] = 1


In [ ]:

data['NO1'].unique()

array([0, 1])

In [ ]:
data_list = []
# Assuming 'data' is your DataFrame
for q, label in zip(data['Column'], data['NO1']):
    entry = []
    entry.append(q)
    entry.append(str(label))
    data_list.append(entry)

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 23)

In [ ]:
print(dataset_train)

[['내담자: 친구도 없고 외로워서 그랬는데, 이렇게 살아가기가 너무 힘들어요. 어떻게 하면 좋을까요?', '0'], ['상담자: 안녕하세요. 무엇을 도와드릴까요? 많이 힘드신 것 같아 걱정이 됩니다.', '0'], ['내담자: 감사합니다. 오늘 제 이야기를 들어주셔서 정말 고마워요. 혼자 끙끙 앓고 있었거든요. 상담자님 덕분에 조금은 살 힘이 생기는 것 같아요. 상담전화로 연락도 해 볼게요. 지금 당장은 막막하지만 포기하지 않고 바깥 세상과 소통하면서 앞으로 나아가 볼게요. 언젠가는 행복도 찾을 수 있겠죠?', '0'], ['상담자: 자살이나 범죄는 절대 해답이 될 수 없어요. 지금은 막막하시겠지만 희망을 잃지 마세요. 반드시 돌파구가 있을 겁니다.', '0'], ['내담자: 전문가 상담을 받아봐야겠어요. 자살 충동이 너무 강해서 걱정이에요.', '0'], ['상담자: 앞으로도 함께 고민하며 걸어가겠습니다. 포기하지 마시고 한 걸음씩 나아가세요. 응원할게요.', '0'], ['상담자: 어린 나이에 부모님 이혼으로 많이 혼란스러웠을 텐데, 진로 결정을 위해 미국까지 가셨다니 정말 고생이 많으셨을 거예요.', '0'], ['내담자: 안녕하세요. 저는 청년내일채움공제에 가입해서 열심히 일하고 있었어요. 2년 뒤면 천이백만원이라는 목돈을 마련할 수 있어서 엄마 빚도 갚고 새 출발할 수 있을 거라 기대했죠. 근데 연휴 바로 전날, 갑자기 대표가 구두로 해고 통보를 하더라고요. 세상이 무너지는 기분이었어요. 엄마 생각에 심장이 먹먹해지고... 15일까지만 출근하라는데 그것도 너무 두려워요. 회사가 워낙 작아서 과장이 횡포를 부리거든요. 전 무시당하고 괴롭힘 당하면서도 돈 때문에 버텼는데, 이제 그 이유마저 사라졌으니 절망적이에요.', '0'], ['상담자: 안녕하세요. 무엇을 도와드릴까요? 요즘 많이 힘드신 것 같네요. 어떤 일이 있으신가요?', '0'], ['내담자: 네, 맞아요. 너무 스트레스 받아요. 그 동료가 저한테만 그러는 건 아니고 모두에게 그래서 더 힘들어

In [ ]:
print(dataset_test)

[['내담자: 제가 힘들어하는 게 남들보다 심각한 건 아닌 것 같아서, 괜히 귀찮게 할 순 없잖아요. 상담 받으러 가려면 또 아빠한테 혼날 것 같기도 하고... 그냥 이대로 있다가 언젠가 죽어버리면 편할 텐데. 계속 살아야 할 이유가 없는 것 같아요.', '1'], ['내담자: 맞아요. 세상에 점 하나 없어진다고 뭐가 달라지겠어요? 그걸 아니까 차라리 가려고 해요. 놓아야 할 걸 못 놓고 있어요. 제 인생에 뭐 하나 이룬 게 없는 것 같아요. 그래서 더 비참해요. 이 나이 먹도록 헛살았다는 생각밖에 안 들어요.', '1'], ['내담자: 안녕하세요 상담선생님. 저에게는 왜 힘들고 아픈 일들만 일어나는 것 같아요. 열심히 일해도 공사대금을 제때 받지 못하고, 가까운 사람을 도와주다가 오해를 받기도 했죠.', '0'], ['내담자: 정말 제가 무엇을 그렇게 잘못했나요? 세상은 저더러 포기하라는 것 같아요. 화가 나고 우울한 마음뿐입니다.', '0'], ['상담자: 아무리 가까운 사이라도 서로를 존중하며 상처 주는 말은 하면 안 됩니다. 남자친구의 그 말이 얼마나 큰 상처가 되었는지 짐작이 가네요. 하지만 그분의 실수로 자존감을 잃으셨다면 안타까운 일이에요.', '0'], ['내담자: 하지만 친구들도 우울증 있는 저 때문에 힘들어할 거예요. 상담사 선생님도 저 같은 환자 줄어들면 좋으시겠죠. 아무도 절 진심으로 걱정해주는 사람이 없는 것 같아요. 제가 세상에서 사라지는 게 모두를 위해 좋은 일인 것 같네요.', '1'], ['내담자: 고마워요. 오늘 상담을 하면서 죽음 말고 다른 선택지도 있다는 걸 깨달은 것 같아요. 아직도 많이 힘들고 막막하지만, 그래도 조금은 살아볼 용기가 생기네요. 혼자 모든 걸 해결하려 들기보다는 주변에 손을 내밀어 봐야겠어요. 쉽지 않은 길이 될 테지만 포기하지 않고 앞으로 나아가 볼게요. 제 삶에 작은 희망이 생길 수 있기를 바라면서요. 오늘 감사했습니다.', '0'], ['상담자: 지금 죽음까지 생각하실 정도로 많이 힘드시군요. 삶의 

In [ ]:
print(len(dataset_train), len(dataset_test))

3282 821


# 모델 불러오기

In [ ]:
# BERTSentenceTransform 수정
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 하이퍼 파라미터 설정
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [ ]:
#import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2,   # 감정 클래스 수로 조정 긍정/부정 클래스2개
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [ ]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

# 모델 학습

In [ ]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))


  0%|          | 0/52 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.8235136866569519 train acc 0.421875
epoch 1 train acc 0.8202791132478632


  0%|          | 0/13 [00:00<?, ?it/s]

epoch 1 test acc 0.8689450290275762


  0%|          | 0/52 [00:00<?, ?it/s]

...

epoch 2 test acc 0.9144140058055152


  0%|          | 0/52 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.1287740021944046 train acc 0.953125
epoch 3 train acc 0.9217080662393162


  0%|          | 0/13 [00:00<?, ?it/s]

epoch 3 test acc 0.9268867924528301


#Fine tuning


In [ ]:
#My Code
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from transformers import AdamW, get_scheduler
from tqdm import tqdm

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2,   # 감정 클래스 수로 조정 긍정/부정 클래스2개
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        logits = self.classifier(out)
        probabilities = F.softmax(logits, dim=-1)  # Apply softmax activation?
        return probabilities

# Assume bertmodel is defined and loaded appropriately
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Print the model architecture
print(model)

# 손실 함수 및 옵티마이저
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

# Learning rate scheduler
epochs = 10
#train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = 16, num_workers = 5)
num_training_steps = epochs * len(train_dataloader)
scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),  # Warm-up for 10% of training steps
    num_training_steps=num_training_steps,
)

# 파인튜닝 루프
model.train()
for epoch in range(epochs):
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        optimizer.step()
        scheduler.step()


BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

100%|██████████| 206/206 [00:56<00:00,  3.65it/s]


In [ ]:
#Dataset
#token_ids shape: torch.Size([16, 100])
#valid_length shape: torch.Size([16])
#segment_ids shape: torch.Size([16, 100])
for epoch in range(3):
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        # 모델 입력 및 출력 형태 확인
        print("token_ids shape:", token_ids.shape)
        print("valid_length shape:", valid_length.shape)
        print("segment_ids:", segment_ids.shape)

NameError: name 'tqdm' is not defined